In [16]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import re
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anmoljeet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
#sentences = 'lol'
#to merge csv files, use
#copy *.csv merged.csv

In [18]:
dataSet = pd.read_csv('merged.csv', encoding = "latin1", names = ["Sentence", "Intent"])
print(dataSet.head())
intent = dataSet["Intent"]
keyIntent = list(set(intent))
sentences = list(dataSet["Sentence"])
  


                                   Sentence         Intent
0  add the binary attribute state to lumber  add_attribute
1    add an boolean attribute number to pie  add_attribute
2      add a numeric attribute value to dog  add_attribute
3      add a boolean attribute id to belief  add_attribute
4  ghost contains a numeric attribute count  add_attribute


In [19]:
def clean(sentences):
    wordList = []
    for i in sentences:
        subbedSentence = re.sub(r'[^ a-z A-Z 0-9]', " ", i)
        words = word_tokenize(subbedSentence)
        wordList.append([j.lower() for j in words]) 
    return wordList

cleanedSetences = clean(sentences)


In [20]:
def tokens(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    token = Tokenizer(filters = filters)
    token.fit_on_texts(words)
    return token

wordTokens = tokens(cleanedSetences)




In [21]:
def maxLength(words):
    return(len(max(words, key = len)))

vocabSize = len(wordTokens.word_index) + 1
maxLengthz = maxLength(cleanedSetences)


Vocab Size = 365 and Maximum length = 8


In [22]:
def encoder(tokens, words):
    return(tokens.texts_to_sequences(words))

encodedSentences = encoder(wordTokens, cleanedSetences)


In [23]:
def padder(words, maxLen):
    return(pad_sequences(words, maxlen = maxLengthz, padding = "post"))

paddedSentences = padder(encodedSentences, maxLengthz)

In [24]:
def oneHotEncoder(encode):
    oneHot = OneHotEncoder(sparse = False)
    return(oneHot.fit_transform(encode))

In [40]:
intentTokens = tokens(keyIntent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
tokenOutput = encoder(intentTokens, intent)
tokenOutput = np.array(tokenOutput).reshape(len(tokenOutput), 1)
oneHotOutput = oneHotEncoder(tokenOutput)

xTrain, xValid, yTrain, yValid = train_test_split(paddedSentences, oneHotOutput, shuffle = True, test_size = 0.15)

In [152]:

def create_model(vocabSizez, maxLen, input_shape):

    model = Sequential()
    model.add(Embedding(vocabSizez, 128))
    model.add(Bidirectional(LSTM(128)))
    #model.add(Dense(128, activation = "relu"))
    #model.add(Dropout(0.5))
    model.add(Dense(64, activation = "relu"))
    model.add(Dropout(0.5))

    model.add(Dense(32, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation = "softmax"))
  
    return model

In [153]:
input_shape = xTrain.shape
print(input_shape)
model = create_model(vocabSize, maxLengthz, input_shape)
model.summary()
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
#model.summary()

modelFileName = 'modelLSTM.h5'
checkpoint = ModelCheckpoint(modelFileName, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(xTrain, yTrain, epochs = 150, batch_size = 32, validation_data = (xValid, yValid), callbacks = [checkpoint])

model.save(modelFileName)


(2121, 8)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, None, 128)         46720     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_82 (Dense)             (None, 64)                16448     
_________________________________________________________________
dropout_60 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_83 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_61 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_84 (Dense)             (None, 5)                 165       



Epoch 00034: val_loss did not improve from 0.00000
Epoch 35/150
2121/2121 [==============================] - 1s 538us/step - loss: 0.0108 - acc: 0.9953 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00035: val_loss did not improve from 0.00000
Epoch 36/150
2121/2121 [==============================] - 3s 1ms/step - loss: 0.0147 - acc: 0.9948 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00036: val_loss did not improve from 0.00000
Epoch 37/150
2121/2121 [==============================] - 2s 840us/step - loss: 0.0088 - acc: 0.9958 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00037: val_loss did not improve from 0.00000
Epoch 38/150
2121/2121 [==============================] - 2s 722us/step - loss: 0.0059 - acc: 0.9967 - val_loss: 1.1921e-07 - val_acc: 1.0000 0s - loss: 0.

Epoch 00038: val_loss did not improve from 0.00000
Epoch 39/150
2121/2121 [==============================] - 2s 774us/step - loss: 0.0088 - acc: 0.9962 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00039: val_

2121/2121 [==============================] - 1s 486us/step - loss: 0.0110 - acc: 0.9953 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00077: val_loss did not improve from 0.00000
Epoch 78/150
2121/2121 [==============================] - 1s 486us/step - loss: 0.0060 - acc: 0.9972 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00078: val_loss did not improve from 0.00000
Epoch 79/150
2121/2121 [==============================] - 1s 538us/step - loss: 0.0040 - acc: 0.9991 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00079: val_loss did not improve from 0.00000
Epoch 80/150
2121/2121 [==============================] - 1s 685us/step - loss: 0.0074 - acc: 0.9967 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00080: val_loss did not improve from 0.00000
Epoch 81/150
2121/2121 [==============================] - 1s 583us/step - loss: 0.0056 - acc: 0.9981 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00081: val_loss did not improve from 0.00000
Epoch 82/150
2121/2121 [===================

2121/2121 [==============================] - 1s 523us/step - loss: 0.0150 - acc: 0.9962 - val_loss: 5.1446e-04 - val_acc: 1.0000

Epoch 00119: val_loss did not improve from 0.00000
Epoch 120/150
2121/2121 [==============================] - 1s 553us/step - loss: 0.0104 - acc: 0.9986 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00120: val_loss did not improve from 0.00000
Epoch 121/150
2121/2121 [==============================] - 1s 560us/step - loss: 0.0142 - acc: 0.9967 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00121: val_loss did not improve from 0.00000
Epoch 122/150
2121/2121 [==============================] - 1s 634us/step - loss: 0.0164 - acc: 0.9962 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00122: val_loss did not improve from 0.00000
Epoch 123/150
2121/2121 [==============================] - 1s 641us/step - loss: 0.0124 - acc: 0.9981 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00123: val_loss did not improve from 0.00000
Epoch 124/150
2121/2121 [==============

In [108]:
model = load_model('modelLSTM.h5')

In [109]:
def predict(text):
    cleanedSentence = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    testWord = word_tokenize(cleanedSentence)
    testWord = [w.lower() for w in testWord]
    testTokens = wordTokens.texts_to_sequences(testWord)
    
    print(testWord)
    if [] in testTokens: #Check for unknown words
        testTokens = list(filter(None, testTokens))
    
    testTokens = np.array(testTokens).reshape(1, len(testTokens))
    x = padder(testTokens, maxLengthz)
    pred = model.predict_proba(x)
  
    return pred



In [110]:
def getIntent(predicts, intents):
    prediction = predicts[0]
 
    intents = np.array(intents)
    ids = np.argsort(-prediction)
    intents = intents[ids]
    predictions = -np.sort(-prediction)
 
    for i in range(predicts.shape[1]):
        print("%s has confidence = %s" % (intents[i], (predictions[i])))


In [151]:
text = "wire is vessel"
pred = predict(text)
print(getIntent(pred, keyIntent))

['wire', 'is', 'vessel']
create_inheritance has confidence = 1.0
add_attribute has confidence = 0.0
create_composition has confidence = 0.0
create_association has confidence = 0.0
add_class has confidence = 0.0
None
